<a href="https://colab.research.google.com/github/rlaaudrb1104/Ai/blob/WOOK/GCBert_%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0_%EC%9E%90%EB%8F%99%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install optuna
!pip install transformers
!pip install accelerate -U
!pip install transformers[torch] -U
!pip install shap
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_

In [8]:
import numpy as np
import torch
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import transformers
import optuna
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd
import re
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from pandas.core.common import random_state
from datasets import load_metric
from transformers import TrainingArguments, Trainer
import logging
import sklearn.metrics as metric
from transformers import DataCollatorWithPadding
import torch,gc
from sklearn.metrics.pairwise import cosine_similarity
from torch.nn import CrossEntropyLoss
from transformers import AdamW

In [12]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cwe file/MSR_1vs1_23000.csv')
df = df.drop(columns="CWE ID")
#df = df.drop(columns="CVE ID")
df = df.drop(columns="vul")

In [13]:
# 모델 및 토크나이저 로딩
model = AutoModelForSequenceClassification.from_pretrained("microsoft/graphcodebert-base", num_labels=10)
tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
def preprocess(df,file_name):
  #preprocess_df = df.replace(re.compile('(^import.*|^from.*)',re.MULTILINE),"",regex=True) #import,from 없애기
  preprocess_df = df.replace(re.compile('(#.*)', re.MULTILINE),"",regex=True) #주석 한 줄
  preprocess_df = preprocess_df.replace(re.compile('[\'\"]{3}.*?[\'\"]{3}', re.DOTALL),"",regex=True) #주석 여러줄
  preprocess_df = preprocess_df.replace(re.compile('[\n]{2,}', re.MULTILINE),"\n",regex=True) #다중개행 한번으로
  preprocess_df = preprocess_df.replace(re.compile('[ ]{4}', re.MULTILINE),"\t",regex=True) #tab 변환
  preprocess_df = preprocess_df.replace(re.compile('[ ]{1,3}', re.MULTILINE)," ",regex=True) #공백 여러개 변환
  preprocess_df.to_csv(file_name)

def tokenized(examples):
  return tokenizer(examples['CODE'], padding=True,truncation=True, return_token_type_ids=True)

In [15]:
# 데이터셋 로딩
preprocess(df,"preprocess.csv")
dataset = load_dataset("csv", data_files="preprocess.csv")['train']
encoded_dataset = dataset.map(tokenized, remove_columns=['CODE'], batched=True)
encoded_dataset = encoded_dataset.rename_column(original_column_name='IDX', new_column_name='labels')
encoded_dataset = encoded_dataset.train_test_split(0.3, seed=100)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/23832 [00:00<?, ? examples/s]

In [16]:
# 데이터 콜레이터
data_collator = transformers.DataCollatorWithPadding(tokenizer=tokenizer)

In [17]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained("microsoft/graphcodebert-base", num_labels=10)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    return {"precision": precision, "recall": recall, "f1": f1}


In [19]:
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 3, 10),
        "seed": trial.suggest_int("seed", 1, 40),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32]),
    }

# 트레이닝 아규먼트 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    report_to="none"
)

# 트레이너 설정
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Optuna와 트레이너 통합
best_run = trainer.hyperparameter_search(direction="maximize", n_trials=10, hp_space=hp_space)

# 최적의 하이퍼파라미터 출력
print("Best run:")
print(best_run)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[I 2024-04-14 15:43:29,187] A new study created in memory with name: no-name-f8facab9-f372-44e2-8854-025f24b47b67
Some we

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.926800,0.809668,0.717396,0.745035,0.710349
2,0.640800,0.644228,0.795028,0.806573,0.788547
3,0.423700,0.490398,0.852741,0.860000,0.854126
4,0.283000,0.483999,0.870802,0.870490,0.866790
5,0.196500,0.442336,0.889383,0.890629,0.888230
6,0.144500,0.428910,0.892912,0.894545,0.892705


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2024-04-14 16:39:20,618] Trial 0 finished with value: 2.6801630996187815 and parameters: {'learning_rate': 1.4546488481662307e-05, 'num_train_epochs': 6, 'seed': 17, 'per_device_train_batch_size': 16}. Best is trial 0 with value: 2.6801630996187815.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batc

OutOfMemoryError: CUDA out of memory. Tried to allocate 48.00 MiB. GPU 0 has a total capacity of 15.77 GiB of which 26.38 MiB is free. Process 23549 has 15.74 GiB memory in use. Of the allocated memory 15.15 GiB is allocated by PyTorch, and 231.30 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)